In [1]:
'''
Notebook for the main text's FIGS?: Simulating cell cycle noise with variable cell volumes and its interplay with the Punisher's performance, deterministically
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

/tmp/ipykernel_21192/2266302094.py:21: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit - WITH HYBRID SIMULATION SUPPORT
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het, \
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, circuit_v = cellmodel_auxil.add_circuit(
    circuits.punisher_cnc_b_initialise,
    circuits.punisher_cnc_b_ode,
    circuits.punisher_cnc_b_F_calc,
    circuits.punisher_cnc_b_eff_m_het_div_k_het,
    par, init_conds,
    # propensity calculation function for variable-volume simulations
    circuits.punisher_cnc_b_v_varvol, varvol=True)

In [3]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch'] = 0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch'] / 80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0  # 0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
par['a_cat'] = 500.0  # promoter strength (unitless)
par['n_cat'] = 300.0
# synthetic protease gene
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0  # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2  # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# plasmid copy number control
init_conds['cat_pb'] = 10.0  # INITIAL CONDITION (not a parameter): all plasmids have working CAT gene copies
par['k_tr'] = 130.2  # plasmid replication rate (1/h)
par['a_inh'] = 948  # inhibitor synthesis rate per plasmid copy (1/h)
par['b_inh'] = 74.976  # inhibitor degradation rate (1/h)
par['n_inh'] = 10  # number of steps of replication initiation at which inhibition can happen
par['K_inh'] = 214.05  # replication inhibition constant (nM)

# critical cell volume triggering division
par['V_crit'] = 2.0 * np.log(2)  # 2ln(2) so as to have an average volume of 1 um^3 assuming constant growth rate

# BURDENSOME GENE REPLICATION
par['mean_rep_phase_b'] = 0.5  # mean replication phase
par['stdev_rep_phase_b'] = 0.23  # standard deviation of replication phase (void as considering avergae dynamics for now)
# scaling factor, makes average burdensome gene concentration equal to that determined by the fixed-volume model
a_b_scale = (par['V_crit']/2) * np.log(2) * np.exp(par['mean_rep_phase_b']*np.log(2))

# culture medium
nutr_qual = 0.5
par['s'] = nutr_qual  # nutrient quality (unitless)
init_conds['s'] = nutr_qual  # nutrient quality (unitless)

In [18]:
# SPECIFY THE SIMULATED SCENARIO
# NO BURDENSOME GENE LOSS (A-D)
loss=False
# BURDENSOME GENE LOSS (E-H)
# loss=True

In [5]:
# FIXED-VOLUME SIMULATION TO FIND THE STARTING STEADY STATE - SET SIMULATION PARAMETERS
# set simulation parameters
tf = (0, 50)  # simulation time frame
savetimestep = 0.1  # save time step
dt_max = 0.1  # maximum integration step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

In [6]:
# FIXED-VOLUME SIMULATION TO FIND THE STARTING STEADY STATE - RUN

sol = ode_sim(par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts_fixedvol = np.array(sol.ts)
xs_fixedvol = np.array(sol.ys)
# det_steady_x = jnp.concatenate((sol.ys[-1, 0:8], jnp.round(sol.ys[-1, 8:])))
det_steady_x = sol.ys[-1, :]

In [19]:
# VARIABLE-VOLUME SIMULATION WITH VARIABLE CELL VOLUME - SET SIMULATION PARAMETERS
if(loss):
    tf_preloss = (tf[-1], tf[-1] + 5)  # simulation time frame before synthetic gene loss
    tf_afterloss = (tf_preloss[-1], tf_preloss[-1] + 35)  # simulation time frame after synthetic gene loss
else:
    tf_preloss = (tf[-1], tf[-1] + 1)  # simulation time frame before synthetic gene loss
    tf_afterloss = (tf_preloss[-1], tf_preloss[-1] + 7)  # simulation time frame after synthetic gene loss
tau = 1e-7  # simulation time step
tau_odestep = 1e-7  # number of ODE integration steps in a single tau-leap step (smaller than tau)
tau_savetimestep = 1e-1  # save time step a multiple of tau

In [6]:
# VARIABLE-VOLUME SIMULATION WITH VARIABLE CELL VOLUME - RUN
mRNA_count_scales, S, x0_tauleap, circuit_synpos2genename, keys0, \
        rep_phase_means_stdevs_bounds = tauleap_sim_prep_varvol(par, len(circuit_genes),
                                                                len(circuit_miscs),
                                                                circuit_name2pos,
                                                                det_steady_x,
                                                                key_seeds=[0]
                                                                )
x0_tauleap[6]=1.0 # start at the default volume of 1 um^3
par['a_b']=par['a_b']*a_b_scale # apply the scaling factor to the burdensome gene transcription rate
ts_jnp_preloss, xs_jnp_preloss, final_keys_preloss = tauleap_sim_varvol(par,  # dictionary with model parameters
                                         circuit_v,  # circuit reaction propensity calculator
                                         circuit_eff_m_het_div_k_het,
                                         x0_tauleap,     # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                         len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                         cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes), # synthetic gene parameters for calculating k values
                                         tf_preloss, tau, tau_odestep, tau_savetimestep,    # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                         mRNA_count_scales, S, circuit_synpos2genename, # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                         keys0, rep_phase_means_stdevs_bounds,
                                         avg_dynamics=True)  # starting random number genereation key

# concatenate the results with the deterministic simulation
ts_preloss = np.concatenate((ts_fixedvol, np.array(ts_jnp_preloss)))
xs_first_preloss = np.concatenate((xs_fixedvol, np.array(xs_jnp_preloss[1])))  # getting the results from the first random number generator key in vmap
xss_preloss = np.concatenate((xs_fixedvol * np.ones((keys0.shape[0], 1, 1)), np.array(xs_jnp_preloss)),axis=1)  # getting the results from all vmapped trajectories

# simulate after synthetic gene expression loss (if there is no loss)
if(loss):
    par['func_b'] = 0.0  # burdensome gene no longer present
ts_jnp, xs_jnp, final_keys = tauleap_sim(par,  # dictionary with model parameters
                                         circuit_v,  # circuit reaction propensity calculator
                                         circuit_eff_m_het_div_k_het,
                                         xs_jnp_preloss[:,-1,:],
                                         # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                         len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                         cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
                                         # synthetic gene parameters for calculating k values
                                         tf_afterloss, tau, tau_odestep, tau_savetimestep,
                                         # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                         mRNA_count_scales, S, circuit_synpos2genename,
                                         # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                         keys0=final_keys_preloss,
                                         avg_dynamics=False)  # starting random number genereation key
# concatenate the results with the deterministic simulation
ts = np.concatenate((ts_preloss, np.array(ts_jnp)))
xs_first = np.concatenate((xs_first_preloss, np.array(xs_jnp[1])))  # getting the results from the first random number generator key in vmap
xss = np.concatenate((xss_preloss, np.array(xs_jnp)), axis=1)  # getting the results from all vmapped trajectories

In [8]:
# xs_first=np.load('../xs_first_fluct_varvol15.npy')
# ts=np.load('../ts_fluct_varvol15.npy')
# xss=np.load('../xss_fluct_varvol15.npy')

In [9]:
# GET MOLECULE CONCENTRATIONS FOR PLOTTING
Vs = xs_first[:, 6]  # cell volumes
xs_concs = np.divide(xs_first, (Vs * np.ones_like(np.array([xs_first[0, :]]).T)).T)  # divide abundances by cell volumes to get concentrations
xs_concs[:, 6] = par['s'] * np.ones_like(Vs)  # instead of volumes, x without variable volumes has nutrient quality in this position

In [10]:
# PLOT THE CELL MODEL TRAJECTORIES
plot_tspan=(tf_preloss[0], tf_afterloss[1])

# PLOT - HOST CELL MODEL
mass_fig = cellmodel_auxil.plot_protein_masses(ts, xs_first, par, circuit_genes)  # plot simulation results
nat_mrna_fig, nat_prot_fig, nat_trna_fig, h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs_first, par,
                                                                                             circuit_genes,
                                                                                             tspan=plot_tspan,
                                                                                             varvol=True)  # plot simulation results
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables(ts,
                                                                                                       xs_first,
                                                                                                       par,
                                                                                                       circuit_genes,
                                                                                                       circuit_miscs,
                                                                                                       circuit_name2pos,
                                                                                                       circuit_eff_m_het_div_k_het,
                                                                                                       tspan=plot_tspan,
                                                                                                       varvol=True)  # plot simulation results
vol_figure = cellmodel_auxil.plot_volume(ts, xs_first, par, circuit_genes, tspan=plot_tspan)  # plot simulation results
bkplot.show(bklayouts.grid([[nat_mrna_fig, nat_prot_fig, vol_figure],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, D_figure]]))

In [11]:
# PLOT THE CIRCUIT TRAJECTORIES

# bkplot.output_file(filename="circuit_sim.html",
#                    title="Synthetic Gene Circuit Simulation")  # set up bokeh output file
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs_first, par,
                                                                                   circuit_genes,
                                                                                   circuit_miscs,
                                                                                   circuit_name2pos,
                                                                                   circuit_styles, tspan=plot_tspan,
                                                                                   varvol=True)  # plot simulation results
F_fig = cellmodel_auxil.plot_circuit_regulation(ts, xs_first, circuit_F_calc, par, circuit_genes, circuit_miscs,
                                                circuit_name2pos, circuit_styles, tspan=plot_tspan,
                                                varvol=True)  # plot simulation results
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig],
                            [F_fig, None, None]]))

In [20]:
# MAKE FIGURE A OR E: CELL GROWTH RATE

# get the growth rate for plotting
_, ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs_concs, par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
if(loss):
    x_axis_label = "Time since mutation, h"
else:
    x_axis_label = "Time since reference, h"

# initialise
fig_ae = bkplot.figure(
    frame_width=200,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="Cell growth rate, 1/h",
    x_range=(tf_preloss[0]-tf_preloss[1],tf_afterloss[1]-tf_afterloss[0]),
    y_range=(0.25,1.75),
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_ae.output_backend = "svg"

if(loss):
    # add shading to show when synthetic gene expression loss occurs
    fig_ae.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,2,2,0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_ae.add_layout(bkmodels.Label(x=0, y=1.75,
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))

# plot the growth rate
fig_ae.line(ts-tf_afterloss[0],np.array(ls), line_width=1.5, line_color=bkRGB(0,0,0))

# set fonts
fig_ae.xaxis.axis_label_text_font_size = "8pt"
fig_ae.xaxis.major_label_text_font_size = "8pt"
fig_ae.yaxis.axis_label_text_font_size = "8pt"
fig_ae.yaxis.major_label_text_font_size = "8pt"

# set ticks
fig_ae.yaxis.ticker=np.arange(0.25, 1.76, 0.25)

# show the plot
bkplot.show(fig_ae)

In [23]:
# MAKE FIGURE B OR F: CAT GENE CONCENTRATIONS

if(loss):
    x_axis_label = "Time since mutation, h"
else:
    x_axis_label = "Time since reference, h"
    
# initialise
fig_bf_y_range = (0, 15)
fig_bf = bkplot.figure(
    frame_width=200,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="Plasmid conc., nM",
    x_range=(tf_preloss[0]-tf_preloss[1],tf_afterloss[1]-tf_afterloss[0]),
    y_range=fig_bf_y_range,
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_bf.output_backend = "svg"

# if simulating burdensome gene mutation
if(loss):
    # add shading to show when synthetic gene expression loss occurs
    fig_bf.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,fig_bf_y_range[1],fig_bf_y_range[1],0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_bf.add_layout(bkmodels.Label(x=0, y=fig_bf_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))

# plot the plasmid concentrations
fig_bf.line(ts - tf_afterloss[0], xs_concs[:, circuit_name2pos['cat_pb']], line_width=1.5,
             line_color=bkRGB(100, 149, 237), legend_label="ccat")
fig_bf.line(ts - tf_afterloss[0], xs_concs[:, circuit_name2pos['cat_lri1']], line_width=1.5,
             line_color=bkRGB(127, 255, 212), legend_label="cLRi")
fig_bf.line(ts - tf_afterloss[0], xs_concs[:, circuit_name2pos['no_cat']], line_width=1.5,
             line_color=bkRGB(207, 181, 59), legend_label="cno")

# legend settings
fig_bf.legend.location = "bottom_right"
fig_bf.legend.label_text_font_size="8pt"
fig_bf.legend.background_fill_alpha = 1
fig_bf.legend.margin = 4
fig_bf.legend.padding = 2
fig_bf.legend.spacing = 2
fig_bf.legend.glyph_width = 10

# set fonts
fig_bf.xaxis.axis_label_text_font_size = "8pt"
fig_bf.xaxis.major_label_text_font_size = "8pt"
fig_bf.yaxis.axis_label_text_font_size = "8pt"
fig_bf.yaxis.major_label_text_font_size = "8pt"

# set ticks
fig_bf.yaxis.ticker=np.arange(0, 16, 2.5)    

# show plot
bkplot.show(fig_bf)

In [24]:
# MAKE FIGURE C OR G: CELL VOLUME

if(loss):
    x_axis_label = "Time since mutation, h"
else:
    x_axis_label = "Time since reference, h"
    
# initialise
fig_cg = bkplot.figure(
    frame_width=200,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="Cell volume, fL",
    x_range=(tf_preloss[0]-tf_preloss[1],tf_afterloss[1]-tf_afterloss[0]),
    y_range=(0.5, 1.5),
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_cg.output_backend = "svg"

if(loss):
    # add shading to show when synthetic gene expression loss occurs
    fig_cg.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,2,2,0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_cg.add_layout(bkmodels.Label(x=0, y=1.75,
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))

# plot the cell volumes
fig_cg.line(ts-tf_afterloss[0], Vs, line_width=1.5, line_color=bkRGB(0,0,0))

# set fonts
fig_cg.xaxis.axis_label_text_font_size = "8pt"
fig_cg.xaxis.major_label_text_font_size = "8pt"
fig_cg.yaxis.axis_label_text_font_size = "8pt"
fig_cg.yaxis.major_label_text_font_size = "8pt"

# set ticks
fig_cg.yaxis.ticker=np.arange(0.75, 1.76, 0.25)
# show the plot
bkplot.show(fig_cg)

In [26]:
if(loss):
    fig_bf_y_range_m_b=(0, 2e4)
    fig_bf_y_range_p_b=(0,2.51e5)
    x_axis_label = "Time since mutation, h"
else:
    fig_bf_y_range_m_b=(1e4, 2e4)
    fig_bf_y_range_p_b=(1.0e5,2.51e5)
    x_axis_label = "Time since reference, h"

# y range for the plot (in terms of burdensome MRNA conc.)
fig_dh_y_range = (fig_bf_y_range_m_b[0], fig_bf_y_range_m_b[1])

# initialise
fig_dh = bkplot.figure(
    frame_width=200,
    frame_height=100,
    x_axis_label="Time since mutation, h",
    y_axis_label="Burdensome gene\nmRNA conc., nM",
    x_range=(tf_preloss[0]-tf_preloss[1],tf_afterloss[1]-tf_afterloss[0]),
    y_range=fig_dh_y_range,
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_dh.output_backend = "svg"

if(loss):
    # add shading to show when synthetic gene expression loss occurs
    fig_dh.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[fig_dh_y_range[0],fig_dh_y_range[1],fig_dh_y_range[1],fig_dh_y_range[0]],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_dh.add_layout(bkmodels.Label(x=0, y=fig_dh_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))

# settings for the main y-axis (for m_b)
fig_dh.yaxis.axis_line_color=bkRGB(252, 194, 0)
fig_dh.yaxis.major_tick_line_color=bkRGB(252, 194, 0)
fig_dh.yaxis.minor_tick_line_color=bkRGB(252, 194, 0)

# plot the cat protein concentrations
fig_dh.line(ts-tf_afterloss[0],xs_concs[:,circuit_name2pos['m_b']], line_width=2, line_color=bkRGB(252, 194, 0), legend_label="mb")

# create an extra  y range for plotting burdensome PROTEIN concentrations
fig_dh.extra_y_ranges = {"p_b": bkmodels.Range1d(start=fig_bf_y_range_p_b[0], end=fig_bf_y_range_p_b[1])}
fig_dh.add_layout(bkmodels.LinearAxis(y_range_name="p_b",
                                     axis_label="Burdensome gene\nprot. conc., nM",
                                     axis_line_color=bkRGB(187, 51, 133),
                                     major_tick_line_color=bkRGB(187, 51, 133),
                                     minor_tick_line_color=bkRGB(187, 51, 133)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
fig_dh.line(ts-tf_afterloss[0],xs_concs[:,circuit_name2pos['p_b']], line_width=2, line_color=bkRGB(187, 51, 133), y_range_name="p_b", legend_label="pb")

# add legend
fig_dh.legend.location = "bottom"
fig_dh.legend.orientation = "horizontal"
fig_dh.legend.label_text_font_size="8pt"
# fig_dh.legend.background_fill_alpha = 1
# fig_dh.legend.border_line_color = 'gray'
# fig_dh.legend.border_line_width = 1
fig_dh.legend.margin = 2
fig_dh.legend.padding = 2
fig_dh.legend.spacing = 2
fig_dh.legend.glyph_width = 10

# set fonts
fig_dh.xaxis.axis_label_text_font_size = "8pt"
fig_dh.xaxis.major_label_text_font_size = "8pt"
fig_dh.yaxis.axis_label_text_font_size = "8pt"
fig_dh.yaxis.major_label_text_font_size = "8pt"

# ticks formatting
fig_dh.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")
fig_dh.yaxis[1].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# show plots
bkplot.show(fig_dh)